In [28]:
import numpy as np
import scipy.sparse as sp

In [29]:
#5)
offsets = [1, 0, -1]
n = 1000
v = np.ones( (1,n) )
data = np.array([[1, 4, 1]]).T @ v
A = sp.dia_matrix( (data, offsets), shape=(n,n) )
b = n * np.ones(n)
b[0]=5; b[-1] = 5
x0 = np.zeros(n)

# (CGM(A, b, x0, tol))

# Computer problem 2.5.1)

In [30]:
def jacobi_method(A, b, x_guess, tol):
    n = len(b)
    x_correct = np.ones(n)
    diag = np.diag(A)
    r = A - np.diag(diag)  # reminder matrix without diagonal
    back_err = np.max(np.abs(x_guess - x_correct))
    steps = 0
    while back_err > tol:
        steps += 1
        x_guess = (b-r.dot(x_guess))/diag
        back_err = np.max(np.abs(x_guess - x_correct))

    return back_err, steps


In [31]:
offsets = [1.0, 0.0, -1.0]
n = 100
v = np.ones( (1,n) )
data = np.array([[-1, 3, -1]]).T @ v
A = sp.dia_matrix( (data, offsets), shape=(n,n) )
b = np.ones(n)
b[0], b[-1] = 2.0, 2.0

x_guess = np.zeros(n)

back_err, step = (jacobi_method(A.toarray(), b, x_guess, 1e-6))
print("Backward error: ", back_err)
print(step, "steps")

Backward error:  6.867614882333228e-07
35 steps


# Computer problem 2.6)
**1)**

In [32]:
#2.5.1) Conjugate Gradient Method

def CGM(A, b, x0, tol):
    (n, n) = A.shape
    x = []; r = []; d=[]
    alpha = []; beta = []
    x.append(x0)
    r.append(b-A@x[0])
    d.append(r[0])
    for k in range(n):
        if np.linalg.norm(r[k]) < tol:
            print(f"Ferdig etter {k} iterasjoner")
            break
        alpha.append( (r[k]).dot(r[k])/ (d[k].dot(A.dot(r[k]))))
        x.append(x[k] + alpha[k]*d[k])
        r.append(r[k] - alpha[k]*A.dot(d[k]))
        beta.append( (r[k+1].dot(r[k+1])) / (r[k].dot(r[k])) )
        d.append(r[k+1] + beta[k]*d[k])
    return x[-1], k, r[-1]

In [33]:
# 1a)
A1 = np.array([[1, 0], [0, 2]])
b1 = np.array([2.0, 4])
A2 = np.array([[1, 2], [2, 5]])
b2= np.array([1, 1])

x0 = [1, 0]
tol = 2.0**(-52)

x1, k1, r1 = CGM(A1, b1, x0, tol)
x2, k2, r2 = CGM(A2, b2, x0, tol)

print(x1)
print(x2)

[2. 2.]
[ 3. -1.]


**5)**

In [34]:
offsets = [1.0, 0.0, -1.0]
n = 10000
v = np.ones( (1,n) )
data = np.array([[-1, 3, -1]]).T @ v
A = sp.dia_matrix( (data, offsets), shape=(n,n) )
A = A.toarray()

for i in range(n):
    if A[i][n-(i+1)] == 0:
        A[i][n-(i+1)] = 1/2

b = 1.5 * np.ones(n)
b[0], b[n-1] = 2.5, 2.5

if (n%2) == 0:
    b[int(n/2)] = 1
    b[int(n/2 -1)] = 1
else:
    b[int(n/2)] = 1
    b[int(n/2 + 1)] = 1


x0 = np.zeros(n)
tol = 2.0**(-52)

print(A)

x, k, r = CGM(A, b, x0, tol)
print(f"Final residual: {r}")
print(f"Step length: {k}")

[[ 3.  -1.   0.  ...  0.   0.   0.5]
 [-1.   3.  -1.  ...  0.   0.5  0. ]
 [ 0.  -1.   3.  ...  0.5  0.   0. ]
 ...
 [ 0.   0.   0.5 ...  3.  -1.   0. ]
 [ 0.   0.5  0.  ... -1.   3.  -1. ]
 [ 0.5  0.   0.  ...  0.  -1.   3. ]]
Ferdig etter 34 iterasjoner
Final residual: [8.48279775e-25 1.97931900e-24 3.77013702e-24 ... 3.77013702e-24
 1.97931900e-24 8.48279775e-25]
Step length: 34


# Cholesky

In [35]:
def Cholesky_transform (A) :
    shape_of_A = A.shape
    R=np.zeros( shape_of_A )
    (n,n)=shape_of_A
    for k in range(n) :
        if A[k,k]<0: raise ValueError ('A is not positivedefinite')
        R[k,k] = np.sqrt(A [k ,k])
        u=np.array ([A[k, k + 1:]/R[k,k]])
        R[k, k+1:n]=u
        A[k+1:n, k+1:n] = A[k+1:n, k+1:n]-u.T@u
    return R

In [36]:
A = np.array([[4, 2], [2, 3]])
print(Cholesky_transform(A))
B = np.array([[3, 2, 1], [2, 6, 2], [1, 2, 3]])
print(Cholesky_transform(B))

[[2.         1.        ]
 [0.         1.41421356]]
[[1.73205081 1.15470054 0.57735027]
 [0.         2.         0.5       ]
 [0.         0.         1.        ]]
